<a href="https://colab.research.google.com/github/VictorSilvaCamargo/Formas_Normais/blob/main/FormasNormais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# biblioteca para imprimir em latex
from IPython.display import display, Latex
# Biblioteca para  realizar operações de busca, correspondência, substituição e manipulação de texto com base em padrões específicos
import re

# Definir uma função para formatar uma expressão lógica substituindo os operadores
def expressao_formatada(expressao):
    # Substituir os operadores lógicos LaTeX por notação mais convencional
    expressao_formatada = expressao.replace(r"\land", r" \wedge ")
    expressao_formatada = expressao_formatada.replace(r"\lor", r" \vee ")
    expressao_formatada = expressao_formatada.replace(r"\neg", r" \neg ")
    expressao_formatada = expressao_formatada.replace(r"\rightarrow", r" \Rightarrow ")
    expressao_formatada = expressao_formatada.replace(r"\leftrightarrow", r" \Leftrightarrow ")
    return expressao_formatada

# Definir uma função para avaliar uma expressão lógica com valores de variáveis fornecidos
def evaluate_expressao(expressao, valores_variáveis):
    # Substituir variáveis pelos valores fornecidos
    for var, valor in valores_variáveis.items():
        expressao = expressao.replace(var, str(int(valor)))
    # Substituir operadores LaTeX por notação mais convencional
    expressao = expressao.replace(r"\land", " and ")
    expressao = expressao.replace(r"\lor", " or ")
    expressao = expressao.replace(r"\neg", " not ")
    expressao = expressao.replace(r"\Rightarrow", "<=")

    # Corrigir a equivalência lógica ("<=>") para "==" dentro de parênteses
    expressao = re.sub(r'\(([^()]+)==([^()]+)\)', r'(\1 == \2)', expressao)

    # Corrigir a dupla implicação ("\Leftrightarrow") para "==" dentro de parênteses
    expressao = expressao.replace(r"\Leftrightarrow", " == ")


    return int(eval(expressao))

# Definir uma função para gerar uma tabela verdade para uma expressão lógica
def gerar_tabela_verdade(expressao, mapeamento_variável):
    # Capturar os operadores antes do primeiro parêntese
    operadores_antes_parenteses = re.search(r'([^()]+)\(', expressao)
    operadores = operadores_antes_parenteses.group(1) if operadores_antes_parenteses else ""

    # Remover o que está antes do primeiro "(" da expressão
    expressao = re.sub(r'^[^()]*\(', '(', expressao)

    # Encontrar todas as variáveis na expressão
    variaveis = re.findall(r'[A-Za-z]+\([^)]*\)', expressao)
    variaveis = sorted(set(variaveis))

    # Mapear variáveis complexas para variáveis simples
    expressao_com_vars_complexos = expressao
    for var_simples, var_complexo in mapeamento_variável.items():
        expressao_com_vars_complexos = expressao_com_vars_complexos.replace(var_complexo, var_simples)

    # Calcular o número de variáveis
    num_variaveis = len(variaveis)

    # Cabeçalho da tabela verdade
    cabeca = "   ".join(variaveis) + "   resultado"
    print(cabeca)
    print("-" * (num_variaveis * 4 + 10))

    conjuncoes = []
    dijuncoes = []

    # Gerar todas as combinações de valores para as variáveis
    for i in reversed(range(2 ** num_variaveis)):
        variavel_dict = {}
        for j, var in enumerate(variaveis):
            variavel_dict[var] = (i >> (num_variaveis - 1 - j)) & 1

        # Avaliar a expressão lógica com os valores das variáveis
        resultado = evaluate_expressao(expressao_com_vars_complexos, variavel_dict)

        # Imprimir os valores das variáveis e o resultado
        values_str = "   ".join(str(variavel_dict[var]) for var in variaveis)
        print(f"{values_str}   {resultado}")
        print("\n")

        if resultado:
            conjuncao = r" \land ".join([f"\\neg{var}" if variavel_dict[var] else f"{var}" for var in variaveis])
            conjuncoes.append(conjuncao)
        else:
            dijuncao = r" \lor ".join([f"\\neg{var}" if variavel_dict[var] else f"{var}" for var in variaveis])
            dijuncoes.append(dijuncao)

    # Criar e exibir a conjunção de dijunções formatada
    conjucao_formatada = expressao_formatada(f"{operadores}(" + r" \land ".join(f"({disj})" for disj in dijuncoes) + ")")
    print("Conjunção de dijunções: \n")
    display(Latex(f"${conjucao_formatada}$"))
    print("\n")

    # Criar e exibir a dijunção de conjunções formatada
    dijuncao_formatada = expressao_formatada(f"{operadores}(" + r" \lor ".join(f"({conj})" for conj in conjuncoes) + ")")
    print("Dijunção de conjunções:\n")
    display(Latex(f"${dijuncao_formatada}$"))
    print("\n")

# Ler as expressões lógicas e mapeamento de variáveis do arquivo TXT
with open('FormasNormais.txt', 'r') as file:
    linhas = file.readlines()
    expressoes = []
    mapeamento_variável = {}

    for linha in linhas:
        linha = linha.strip()
        if linha.startswith("Expressao: "):
            expressao = linha.split(": ")[1]
            expressoes.append(expressao)
        elif linha.startswith("Mapeamento: "):
            linha_de_mapeamento = linha.split(": ")[1]
            # Parse o mapeamento de variáveis
            variable_pairs = re.findall(r'(\w+)\s*=\s*(\S+)', linha_de_mapeamento)
            for var_simples, var_complexo in variable_pairs:
                mapeamento_variável[var_complexo] = var_simples

    # Processar cada expressão
    for expressao in expressoes:
        gerar_tabela_verdade(expressao, mapeamento_variável)


P(x)   Q(y)   resultado
------------------
1   1   0


1   0   0


0   1   0


0   0   1


Conjunção de dijunções: 



<IPython.core.display.Latex object>



Dijunção de conjunções:



<IPython.core.display.Latex object>



P(x)   Q(y)   resultado
------------------
1   1   1


1   0   0


0   1   1


0   0   1


Conjunção de dijunções: 



...

<IPython.core.display.Latex object>



P(x)   Q(y)   R(x)   S(x, y)   resultado
--------------------------
1   1   1   1   0


1   1   1   0   1


1   1   0   1   0


1   1   0   0   1


1   0   1   1   0


1   0   1   0   1


1   0   0   1   1


1   0   0   0   0


0   1   1   1   1


0   1   1   0   0


0   1   0   1   1


0   1   0   0   0


0   0   1   1   1


0   0   1   0   0


0   0   0   1   1


0   0   0   0   0


Conjunção de dijunções: 



<IPython.core.display.Latex object>



Dijunção de conjunções:



<IPython.core.display.Latex object>